# Color detection with opencv

In [ ]:
# Import needed packages(libraries)
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv

In [ ]:
# Load your image using cv.imread() , enter your image path with its format(.png/.jpg/...) inside single or double quotation
image = cv.imread()

In [ ]:
# Convert your color space from BGR(default of opencv / loaded image with opencv) to HSV color space using cv.cvtColor()
hsv_image = cv.cvtColor(image,cv.COLOR_BGR2HSV)
print(cv.COLOR_BGR2HSV)
# You can the number 40(key) , that indicates cv.COLOR_BGR2HSV , instead of cv.COLOR_BGR2HSV

In [ ]:
# Show both original and hsv image
# Show in RGB mode with : [...,::-1] or [:,:,::-1]
plt.figure(figsize=[20,20])
plt.subplot(331);plt.imshow(image[...,::-1]);plt.title("Original image");plt.axis("off")
plt.subplot(332);plt.imshow(hsv_image[:,:,::-1]);plt.title("hsv image");plt.axis("off")

# Try with specific lower and upper color of green(optional color) , you try any range

In [ ]:
# Define lower and upper bounds(colors) for color detection(in HSV color space)
# lower_color = np.array([minimum_hue,minimum_saturation,minimum_value])
lower_color = np.array([30,50,50])
# upper_color = np.array([maximum_hue,maximum_saturation,maximum_value])
upper_color = np.array([90,255,255])

In [ ]:
# Create a mask for the specified color range
mask = cv.inRange(hsv_image,lower_color,upper_color)
# Show the mask with specified lower and upper color range
plt.imshow(mask);plt.title("Mask");plt.axis("off")

In [ ]:
# Apply the mask to original image
# The mask will over-load on original image and cover all colors except of where matches to your lower and upper colors
result = cv.bitwise_and(image,image,mask=mask)
# Show result image(combination of your image and created mask)
plt.imshow(result[...,::-1]);plt.title("Result image");plt.axis("off")

In [ ]:
# Display original , mask and result image
cv.imshow("Original image",image)
cv.imshow("Mask",mask)
cv.imshow("Result",result)
cv.waitKey()
cv.destroyAllWindows()

In [ ]:
# Show original , mask  and color detected image(result iamge)
plt.figure(figsize=[20,20])
plt.subplot(331);plt.imshow(image[...,::-1]);plt.title("Original image");plt.axis("off")
plt.subplot(332);plt.imshow(mask);plt.title("Mask");plt.axis("off")
plt.subplot(333);plt.imshow(result[...,::-1]);plt.title("Detected color");plt.axis("off")

# What is hsv ?

In [ ]:
# Create an empty black image with desired size for HSV color bar
bar_width = 360
bar_height = 50
# Create an empty black bar with given width and height
hsv_bar = np.zeros((bar_height,bar_width,3),dtype=np.uint8)
# Show emoty black bar
cv.imshow("HSV bar",hsv_bar)
cv.waitKey()
cv.destroyAllWindows()

In [ ]:
# Fill empty black bar(image) with HSV colors
for x in range(bar_width) :
    # Define hue , saturation and value
    hue = int(x/bar_width*180) # Hue varies from 0 to 180 degrees
    saturation = 255 # Saturation is constant at its maximum value
    value = 255 # Value is constant at its maximum value
    # Create hsv color
    hsv_color = np.array([[[hue,saturation,value]]],dtype=np.uint8)
    # Convert hsv color to bgr color
    rgb_color = cv.cvtColor(hsv_color,cv.COLOR_HSV2BGR)
    # Create bar
    hsv_bar[:,x:x+1] = rgb_color

# Display the hsv bar
cv.imshow("HSV bar",hsv_bar)
cv.waitKey()
cv.destroyAllWindows()

In [ ]:
plt.imshow(hsv_bar[...,::-1]),plt.title("HSV bar");plt.axis("off")

# With the code below , your are able to find the best lower and upper bound you are looking for

In [ ]:
# Define a function to do nothing!!!!
def nothing(x) :
    pass

In [ ]:
# Create a black image
image = np.zeros((100,300,3),dtype=np.uint8)

# Create and name a window containing the created image
cv.namedWindow("image")

# Create trackbars for changing color
# Define minimum parameters
cv.createTrackbar("Minimum hue","image",0,179,nothing)
cv.createTrackbar("Minimum saturation","image",0,255,nothing)
cv.createTrackbar("Minimum value","image",0,255,nothing)

# Define maximum parameters
cv.createTrackbar("Maximum hue","image",0,179,nothing)
cv.createTrackbar("Maximum saturation","image",0,255,nothing)
cv.createTrackbar("Maximum value","image",0,255,nothing)

# Set default value for maximum HSV trackbars
cv.setTrackbarPos("Maximum hue","image",179)
cv.setTrackbarPos("Maximum saturation","image",255)
cv.setTrackbarPos("Maximum value","image",255)

while True :
    # Get current positions of all trackbars
    # Convert your color space from BGR(default of opencv / created image) to HSV color space using cv.cvtColor()
    hsv = cv.cvtColor(image,cv.COLOR_BGR2HSV)
    # Minimum parameters
    minimum_hue = cv.getTrackbarPos("Minimum hue","image")
    minimum_saturation = cv.getTrackbarPos("Minimum saturation","image")
    minimum_value = cv.getTrackbarPos("Minimum value","image")

    # Maximum parameters
    maximum_hue = cv.getTrackbarPos("Maximum hue","image")
    maximum_saturation = cv.getTrackbarPos("Maximum saturation","image")
    maximum_value = cv.getTrackbarPos("Maximum value","image")

    # Create lower and upper color using minimum and maximum parameters
    lower_color = np.array([minimum_hue,minimum_saturation,minimum_value])
    upper_color = np.array([maximum_hue,maximum_saturation,maximum_value])
    # Convert lower and upper range to bgr
    lower_bgr = cv.cvtColor(np.uint8([[lower_color]]),cv.COLOR_BGR2HSV)[0][0]
    upper_bgr = cv.cvtColor(np.uint8([[upper_color]]),cv.COLOR_BGR2HSV)[0][0]
    
    # Create an uniform hsv image
    image[...] = lower_bgr + upper_bgr
    

    # Show image
    cv.imshow("image",image)
    
    if cv.waitKey(1) & 0xFF == 27 :
        # Print last color changes
        # You can remove brackets from printed values(optional)
        print("Lower color : ",str(lower_color).lstrip('[').rstrip(']'))
        print("Upper color : ",str(upper_color).lstrip('[').rstrip(']'))
        # Show the created color
        plt.imshow(image[...,::-1]),plt.title("HSV color");plt.axis("off")
        break

cv.destroyAllWindows()

# Confidence calculator

In [ ]:
# Calculate the confidence of detection
def calculate_color_confidence(frame,lower_color,upper_color,mask) :
    number_of_color_pixels = cv.countNonZero(mask)
    confidence = (number_of_color_pixels / (frame.shape[0] * frame.shape[1])) * 100
    return confidence

In [ ]:
# Load your video using cv.VideoCapture() , enter your video path with its format(.mp4/...) inside single or double quotation
# To use your webcam using cv.VideoCapture(0)
cap = cv.VideoCapture(0)
# Create a window with name of "your window name" , inside single or double quotation using cv.namedWindow("your window name")
cv.namedWindow("Result")

# Create trackbars for changing color
# Define minimum parameters
cv.createTrackbar("Minimum hue","Result",0,179,nothing)
cv.createTrackbar("Minimum saturation","Result",0,255,nothing)
cv.createTrackbar("Minimum value","Result",0,255,nothing)
# Define maximum parameters
cv.createTrackbar("Maximum hue","Result",0,179,nothing)
cv.createTrackbar("Maximum saturation","Result",0,255,nothing)
cv.createTrackbar("Maximum value","Result",0,255,nothing)

# Set default value for maximum HSV trackbars
cv.setTrackbarPos("Maximum hue","Result",179)
cv.setTrackbarPos("Maximum saturation","Result",255)
cv.setTrackbarPos("Maximum value","Result",255)

while True :
    ret , frame = cap.read()
    if not ret :
        break
    # Convert taken frames from video to HSV color space using cv.cvtColor()    
    hsv = cv.cvtColor(frame,cv.COLOR_BGR2HSV)
    # Get current positions of all trackbars
    # Minimum parameters
    minimum_hue = cv.getTrackbarPos("Minimum hue","Result")
    minimum_saturation = cv.getTrackbarPos("Minimum saturation","Result")
    minimum_value = cv.getTrackbarPos("Minimum value","Result")
    # Maximum parameters
    maximum_hue = cv.getTrackbarPos("Maximum hue","Result")
    maximum_saturation = cv.getTrackbarPos("Maximum saturation","Result")
    maximum_value = cv.getTrackbarPos("Maximum value","Result")

    # Create lower and upper color using minimum and maximum parameters
    lower_color = np.array([minimum_hue,minimum_saturation,minimum_value],dtype=np.uint8)
    upper_color = np.array([maximum_hue,maximum_saturation,maximum_value],dtype=np.uint8)

    # Apply the mask to original image
    # The mask will over-load on original image and cover all colors except of where matches to your lower and upper colors
    mask = cv.inRange(hsv,lower_color,upper_color)
    # Create result image(combination of your image and created mask)
    result = cv.bitwise_and(frame,frame,mask=mask)
    # Put some texts
    cv.putText(result,"Result",(10,75),cv.FONT_HERSHEY_PLAIN,4,(0,0,255),4)
    cv.putText(frame,"Tracking",(10,75),cv.FONT_HERSHEY_PLAIN,4,(255,255,255),4)
    
    
    # Find contours in mask
    contours,_ = cv.findContours(mask,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_TC89_KCOS)

    # Create bounding box for each contour
    """
    x is x1 , y is y1 , w(width) is x2 , h(height) is y2
    """
    for contour in contours :
        x,y,w,h = cv.boundingRect(contour)
        cv.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
    #Calculate confidence
    confidence = calculate_color_confidence(frame=frame,lower_color=lower_color,upper_color=upper_color,mask=mask)
    cv.putText(frame,f"Confidence : {confidence:.2f}",(10,160),cv.FONT_HERSHEY_PLAIN,4,(255,0,0),2)
    # Show both frame and result
    cv.imshow("Result",result)
    cv.imshow("Mask",mask)
    cv.imshow("Tracking",frame)
    if cv.waitKey(5) & 0xFF == 27 :
        # Print the latest color changes and confidence
        # You can remove brackets from printed values(optional)
        print("Lower color : ",str(lower_color).lstrip('[').rstrip(']'))
        print("Upper color : ",str(upper_color).lstrip('[').rstrip(']'))
        print(f"Confidence of color detection : {confidence:.2f} %")
        # Show the result and tracked color
        plt.figure(figsize=[20,20])
        plt.subplot(331);plt.imshow(result[...,::-1]);plt.title("Result");plt.axis("off")
        plt.subplot(332);plt.imshow(frame[...,::-1]);plt.title("Tracked color");plt.axis("off")
        plt.subplot(333);plt.imshow(mask);plt.title("Mask");plt.axis("off")
        break

cap.release()
cv.destroyAllWindows()

In [ ]:
# Load your image using cv.imread() , enter your image path with its format(.png/.jpg/...) inside single or double quotation
my_image = cv.imread()
# Create a window with name of "your window name" , inside single or double quotation using cv.namedWindow("your window name")
cv.namedWindow("image")


# Create trackbars for changing color
# Define minimum parameters
cv.createTrackbar("Minimun hue","image",0,179,nothing)
cv.createTrackbar("Minimun saturation","image",0,255,nothing)
cv.createTrackbar("Minimun value","image",0,255,nothing)
# Define maximum parameters
cv.createTrackbar("Maximum hue","image",0,179,nothing)
cv.createTrackbar("Maximum saturation","image",0,255,nothing)
cv.createTrackbar("Maximum value","image",0,255,nothing)

# Set default value for maximum HSV trackbars
cv.setTrackbarPos("Maximum hue","image",179)
cv.setTrackbarPos("Maximum saturation","image",255)
cv.setTrackbarPos("Maximum value","image",255)

while True :
    # Convert image to HSV color space using cv.cvtColor() 
    hsv = cv.cvtColor(my_image,cv.COLOR_BGR2HSV)
    # Get current positions of all trackbars
    # Minimum parameters
    minimum_hue = cv.getTrackbarPos("Minimun hue","image")
    minimum_saturation = cv.getTrackbarPos("Minimun saturation","image")
    minimum_value = cv.getTrackbarPos("Minimun value","image")
    # Maximum parameters
    maximum_hue = cv.getTrackbarPos("Maximum hue","image")
    maximum_saturation = cv.getTrackbarPos("Maximum saturation","image")
    maximum_value = cv.getTrackbarPos("Maximum value","image")
    # Create lower and upper color using minimum and maximum parameters
    lower_color = np.array([minimum_hue,minimum_saturation,minimum_value])
    upper_color = np.array([maximum_hue,maximum_saturation,maximum_value])
    # Apply the mask to original image
    # The mask will over-load on original image and cover all colors except of where matches to your lower and upper colors
    mask = cv.inRange(hsv,lower_color,upper_color)
    # Create result image(combination of your image and created mask)
    result = cv.bitwise_and(my_image,my_image,mask=mask)
    # Calculate confidence
    # frame here means your image
    confidence = calculate_color_confidence(frame=my_image,lower_color=lower_color,upper_color=upper_color,mask=mask)
    # Show the result image
    cv.imshow("image",result)
    if cv.waitKey(5) & 0xFF == 27 :
        # Print last color changes and confidence
        # You can remove brackets from printed values(optional)
        print("Lower color : ",str(lower_color).lstrip('[').rstrip(']'))
        print("Upper color : ",str(upper_color).lstrip('[').rstrip(']'))
        print(f"Confidence of color detection : {confidence:.2f} %")
        # Show the result image
        plt.imshow(result[...,::-1]);plt.title("Result");plt.axis("off")
        cv.destroyAllWindows()
        break

# Detection with involving the confidence

In [ ]:
# Load your video using cv.VideoCapture() , enter your video path with its format(.mp4/...) inside single or double quotation
# Use your webcam using cv.VideoCapture(0)
cap = cv.VideoCapture(0)
# Create a window with name of "your window name" , inside single or double quotation using cv.namedWindow("your window name")
cv.namedWindow("Result")

# Create trackbar for color changing
# Define minimum parameters
cv.createTrackbar("Minimum hue","Result",0,179,nothing)
cv.createTrackbar("Minimum saturation","Result",0,255,nothing)
cv.createTrackbar("Minimum value","Result",0,255,nothing)
# Define maximum parameters
cv.createTrackbar("Maximum hue","Result",0,179,nothing)
cv.createTrackbar("Maximum saturation","Result",0,255,nothing)
cv.createTrackbar("Maximum value","Result",0,255,nothing)

# Set default value for maximum HSV trackbars
cv.setTrackbarPos("Maximum hue","Result",179)
cv.setTrackbarPos("Maximum saturation","Result",255)
cv.setTrackbarPos("Maximum value","Result",255)

while True :
    ret , frame = cap.read()
    if not ret :
        break
    # Convert taken frames from video to HSV color space using cv.cvtColor()    
    hsv = cv.cvtColor(frame,cv.COLOR_BGR2HSV)
    # Get current positions of all trackbars
    # Minimum parameters
    minimum_hue = cv.getTrackbarPos("Minimum hue","Result")
    minimum_saturation = cv.getTrackbarPos("Minimum saturation","Result")
    minimum_value = cv.getTrackbarPos("Minimum value","Result")
    # Maximum parameters
    maximum_hue = cv.getTrackbarPos("Maximum hue","Result")
    maximum_saturation = cv.getTrackbarPos("Maximum saturation","Result")
    maximum_value = cv.getTrackbarPos("Maximum value","Result")

    # Create lower and upper color using minimum and maximum parameters
    lower_color = np.array([minimum_hue,minimum_saturation,minimum_value],dtype=np.uint8)
    upper_color = np.array([maximum_hue,maximum_saturation,maximum_value],dtype=np.uint8)

    # Apply the mask to original image
    # The mask will over-load on original image and cover all colors except of where matches to your lower and upper colors
    mask = cv.inRange(hsv,lower_color,upper_color)
    # Create result image(combination of your image and created mask)
    #Calculate confidence
    confidence = calculate_color_confidence(frame=frame,lower_color=lower_color,upper_color=upper_color,mask=mask)
    # Create result image
    result = cv.bitwise_and(frame,frame,mask=mask)
    # Put some texts
    cv.putText(result,"Result",(10,75),cv.FONT_HERSHEY_PLAIN,4,(0,0,255),4)
    cv.putText(frame,"Tracking",(10,75),cv.FONT_HERSHEY_PLAIN,4,(255,255,255),4)
    
    
    # Find contours in mask
    contours,_ = cv.findContours(mask,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_TC89_KCOS)

    # Create bounding box for each contour
    """
    x is x1 , y is y1 , w(width) is x2 , h(height) is y2
    We use contour area to increase the accuracy of detection
    """
    for contour in contours :
        area = cv.contourArea(contour)
        if 0.37 < confidence < 1 and area < 2300 or area > 4000 :
            # x,y,w,h = cv.boundingRect(contour)
            x,y,w,h = cv.boundingRect(contour)
            cv.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
    cv.putText(frame,f"Confidence : {confidence:.2f}",(10,160),cv.FONT_HERSHEY_PLAIN,4,(255,0,0),2)
    # Show both frame and result
    cv.imshow("Result",result)
    cv.imshow("Mask",mask)
    cv.imshow("Tracking",frame)
    if cv.waitKey(5) & 0xFF == 27 :
        # Print the latest color changes and confidence
        # You can remove brackets from printed values(optional)
        print("Lower color : ",str(lower_color).lstrip('[').rstrip(']'))
        print("Upper color : ",str(upper_color).lstrip('[').rstrip(']'))
        if confidence > 1 :
            print("Confidence is more than 100 %")
            print(f"Confidence of color detection : {confidence:.2f} %")
        else :
            print(f"Confidence of color detection : {confidence:.2f} %")
        # Show the result and tracked color
        plt.figure(figsize=[20,20])
        plt.subplot(331);plt.imshow(result[...,::-1]);plt.title("Result");plt.axis("off")
        plt.subplot(332);plt.imshow(frame[...,::-1]);plt.title("Tracked color");plt.axis("off")
        plt.subplot(333);plt.imshow(mask);plt.title("Mask");plt.axis("off")
        break

cap.release()
cv.destroyAllWindows()

# Problems :
#### The accuracy can be more than this by applying more features to filter detections
#### The red bounding box can be remove when the webcam will open at first
#### This detection is highly depended to brightness of environment , so you can add filters to control this
#### This project doesn't work as the same for all colors , you can manage this issue
#### You can manage what happen is confidence is more than 100 % or 1

# Mohammad Parsa Rezaifar - Computer vision developer